In [ ]:
##A notebook to plot the results for exampleRST-Quarantine

#Dependencies
import matplotlib.pyplot as plt
import pickle
import glob
import numpy as np
import os
import geopandas as gpd
import sys


##Main functions 
def readFile(FileNamePrefix):
    evolutionCases = []
    evolutionTests = []

    if glob.glob(FileNamePrefix + "*.pickle")==[]:
            print('Input files not found')
            sys.exit(1)
    for fname in glob.glob(FileNamePrefix + "*.pickle"):
        with open(fname, "rb") as f: 
            CovidCases, TestingHistory, Symptomatic, Localities = pickle.load(f)
            NumSteps=TestingHistory.shape[1]
            evolutionCases_temp = [np.sum(CovidCases[:,j]) for j in range(NumSteps)]
            evolutionTests_temp = [np.sum(  TestingHistory[TestingHistory[:,j] > 0, j]  )    for j in range(NumSteps)]
            evolutionCases.append(evolutionCases_temp)
            evolutionTests.append(evolutionTests_temp)
            
    return evolutionCases, evolutionTests


## A function to smooth an array inp
def smooth(inp, window):
    
    out=np.zeros(inp.shape) 
    for i in range(inp.shape[1]):
        ind_avg=[i-j  for j in range(min(window, i+1))]
        out[:,i]= np.sum(inp[:,ind_avg],axis=1 )
    return list(out/window)
        

##Plot and save plot
def plotData(evolutionCases, evolutionTests, Filename=None):

    NumSteps=np.array(evolutionCases).shape[1]
    evolutionCasesMean = np.mean(np.array(evolutionCases), axis=0)
    
    

    evolutionTestsMean = np.mean(np.array(evolutionTestsRST), axis=0)
    evolutionTestsStd = np.std(np.array(evolutionTestsRST), axis=0)
    
    #Tests
    fig, ax = plt.subplots()
    

    ax.plot(np.arange(NumSteps),evolutionTestsMean, 'red', label= 'RST')
    ax.fill_between(np.arange(NumSteps), evolutionTestsMean+evolutionTestsStd, evolutionTestsMean-evolutionTestsStd, facecolor="red", alpha=0.1)
    

    ax.set_ylabel('Number of Covid Positive Tests')
    ax.set_xlabel('Day number')
    ax.legend(loc="upper right", bbox_to_anchor=(0.5, 0.4, 0.5, 0.5))
    ax.grid(True)


    #Ground Truth
    ax1=ax.twinx()
    # ax1.errorbar(np.arange(NumSteps), evolutionCasesMean, yerr=evolutionCasesStd)
    #ax1.fill_between(np.arange(NumSteps), evolutionCasesMean+evolutionCasesStd, evolutionCasesMean-evolutionCasesStd, facecolor="blue", alpha=0.3)
    ax1.plot(np.arange(NumSteps),evolutionCasesMean,'--',label="GroundTruth", alpha=0.5)
    ax1.set_ylabel('Number of Ground Truth Covid Cases')
    ax1.set_xlabel('Day number')
    ax1.legend(loc="upper right")
    plt.show()
    if Filename:
        fig.savefig(Filename, bbox_inches="tight")
        
#Plot and save geoplot        
def geoplotData(FileNamePrefix, OutFolder, OutFileSuffix, BangaloreDataFile, DayList, window, GT=False):

    Iter=0

    for fname in glob.glob(FileNamePrefix + "*.pickle"):
        with open(fname, "rb") as f: 
            CovidCases, TestingHistory, Symptomatic, Localities = pickle.load(f)
            if Iter==0:
                TestingHistoryGlobal=np.zeros(TestingHistory.shape)
                CovidCasesGlobal = np.zeros(CovidCases.shape)
            TestingHistoryGlobal+=TestingHistory
            CovidCasesGlobal+=CovidCases
            Iter+=1
        
    TestingHistory=TestingHistoryGlobal/Iter
    CovidCases=CovidCasesGlobal/Iter
        
    #Read Bangalore data 
    Bangalore=gpd.read_file(BangaloreDataFile)

    for day in DayList:
        ind_col=[day-i for i in range(min(window, day+1))]
        TestingCum=np.sum(TestingHistory[:, ind_col]>0, axis=1)

        Bangalore['TotalInfected']=np.zeros(Bangalore.shape[0])
        Bangalore['TotalPositive']=np.zeros(Bangalore.shape[0])

        for i in range(Bangalore.shape[0]):
            Bangalore.loc[i, 'TotalInfected']=CovidCases[int(Bangalore.loc[i, 'wardNo'])-1,day-1]
            Bangalore.loc[i, 'TotalPositive']=sum(TestingCum[Localities==Bangalore.loc[i, 'wardName']])
            
        fig1, ax1=plt.subplots()
        ax1.set_title('Ground Truth on Day '+str(day))
        Bangalore.plot(column="TotalInfected", cmap='OrRd', legend='True', ax=ax1)
        fig2, ax2=plt.subplots()
        ax2.set_title('Positive over the last '+str(window)+' days on Day '+str(day))
        Bangalore.plot(column="TotalPositive", cmap='OrRd', legend='True',ax=ax2)
        if GT==True:
            fig1.savefig(OutFolder+str(day)+'GT_Geo', bbox_inches="tight")
        fig2.savefig(OutFolder+str(day)+OutFileSuffix, bbox_inches="tight")

In [ ]:
##Read Data from the pickle file

DataDirectory = "OutputData/"
ParentDirectory = "example/SimulationResults/"
InputFileNamePrefix=ParentDirectory+DataDirectory+"RandomSymptomaticTesting_UniformSeed_InterventionQuarantine_TestingBudget50_FalseNegative0_Iter_"


evolutionCasesRST, evolutionTestsRST=readFile(InputFileNamePrefix)

In [ ]:
##Plot and save the figure

%matplotlib inline

#Directory for storing output plots
OutputDirectory= "OutputPlots/"
ParentDirectory = "example/SimulationResults/"
OutputFile1=ParentDirectory+OutputDirectory+"RSTFullReportingUSeed"
OutputFile2=ParentDirectory+OutputDirectory+"RSTFullReportingUSeedsmooth"

#Create output directory                                                                                                                                             
path = os.path.join(ParentDirectory, OutputDirectory)
if not os.path.exists(path):
    os.mkdir(path)

plotData(evolutionCasesRST, evolutionTestsRST,OutputFile1)

window=8

evolutionTestsRSTsmooth=smooth(np.array(evolutionTestsRST),window)

plotData(evolutionCasesRST, evolutionTestsRSTsmooth, OutputFile2)




In [ ]:
#Show geographical variation

DayList=[10, 20, 30, 40, 50] #Assuming 100 days simulation
window=8
OutFolder=ParentDirectory+OutputDirectory
OutFileSuffix='RST_Geo'
BangaloreDataFile='example/InputData/city.geojson'

Bangalore = geoplotData(InputFileNamePrefix,OutFolder,OutFileSuffix, BangaloreDataFile, DayList, window, True)